In [1]:
import pandas as pd
import getpass
import sqlalchemy as sa
from sqlalchemy import create_engine

In [2]:
# 1.

driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your password:'"")
ip = '127.0.0.1'  

Your password: ········


In [3]:
connection_string = f'{driver}://{user}:{password}@{ip}'

In [4]:
db_connection = sa.create_engine(connection_string)

In [5]:
# 2.

def rentals_month(month,year,engine):
    
    query = f'SELECT * FROM sakila.rental WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};'
    df = pd.read_sql_query(query, engine)
    
    return df

In [6]:
df = rentals_month(5,2005,db_connection)

In [7]:
df

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


In [8]:
# 3.
'''
def rental_count_month(df):
    df['rental_date'].dt.year[0]
    df['rental_date'].dt.month[0]
    
    year = str(df['rental_date'].dt.year[0])
    month = str(df['rental_date'].dt.month[0])
    
    query = f"SELECT customer_id, count(rental_id) FROM sakila.rental WHERE MONTH(rental_date) = {month} and YEAR(rental_date) = {year} GROUP BY customer_id;"
'''

'\ndef rental_count_month(df):\n    df[\'rental_date\'].dt.year[0]\n    df[\'rental_date\'].dt.month[0]\n    \n    year = str(df[\'rental_date\'].dt.year[0])\n    month = str(df[\'rental_date\'].dt.month[0])\n    \n    query = f"SELECT customer_id, count(rental_id) FROM sakila.rental WHERE MONTH(rental_date) = {month} and YEAR(rental_date) = {year} GROUP BY customer_id;"\n'

In [9]:
year = str(df['rental_date'].dt.year[0])
month = str(df['rental_date'].dt.month[0])

In [10]:
#query = f"SELECT customer_id, count(rental_id) FROM sakila.rental WHERE MONTH(rental_date) = {month} and YEAR(rental_date) = {year} GROUP BY customer_id;"

In [11]:
#df = pd.read_sql_query(query, db_connection)

In [15]:
df_customer = df.groupby(df['customer_id'])[['rental_id']].count()

In [16]:
df_customer

,rental_id
customer_id,
1,2
2,1
3,2
5,3
6,3
...,...
594,4
595,1
596,6


In [17]:
df_customer.rename(columns={"rental_id":f"rentals_{month}_{year}"})

,rentals_5_2005
customer_id,
1,2
2,1
3,2
5,3
6,3
...,...
594,4
595,1
596,6


In [25]:
#df = pd.read_sql_query("SELECT * FROM sakila.rental", db_connection)

In [30]:
def rental_count_month(df, year, month):
    
    df_customer = df.groupby(df['customer_id'])[['rental_id']].count()
    df_customer = df_customer.rename(columns={"rental_id":f"rentals_{month}_{year}"})
    
    return df_customer

In [31]:
df_customer_1 = rental_count_month(df, 2005, 5)

In [32]:
df_customer_1

,rentals_5_2005
customer_id,
1,32
2,27
3,26
4,22
5,38
...,...
595,30
596,28
597,25


In [33]:
df_customer_2 = rental_count_month(df, 2005, 6)

In [34]:
df_customer_2

,rentals_6_2005
customer_id,
1,32
2,27
3,26
4,22
5,38
...,...
595,30
596,28
597,25


In [18]:
# 4.

In [ ]:
def compare_rentals(df1, df2):

In [35]:
df = df_customer_1.join(df_customer_2,on='customer_id')

In [36]:
df

,rentals_5_2005,rentals_6_2005
customer_id,,
1,32,32
2,27,27
3,26,26
4,22,22
5,38,38
...,...,...
595,30,30
596,28,28
597,25,25
